In [13]:
import os

os.environ["SEED"] = "0"

import numpy as np
import pandas as pd

from src.aac.AAC import AAC
from src.constant import DATA_DIR, DATABASE_DIR, MAIN_DIR, SEED
from src.database import DB
from src.database.db import DB_PATH
from src.database.queries import get_model_training_data, get_solvers_count
from src.experiment import parhydra
from src.instance.TSP_Instance import TSP_from_index_file, set_n22_cut_off_time
from src.log import logger
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.surrogate.SurrogatePolicy import (
    EvaluationSurrogatePolicyA,
    EvaluationSurrogatePolicyB,
    EvaluationSurrogatePolicyC,
    IterationSurrogatePolicyA,
    IterationSurrogatePolicyB,
    SurrogatePolicy,
    TestSurrogatePolicy,
)
from src.surrogate.wrapper import EmptyWrapper, SurvivalFunctionWrapper

In [15]:
# db_path_list = list(DATABASE_DIR.glob("run-policy-*.db"))

# records = []

# for db_path in db_path_list:
#     results = DB(db_path).get_results()
#     print(db_path, results.loc[results["prefix"].str.startswith("test")].shape[0])

In [ ]:
db_path_list = list(DATABASE_DIR.glob("run-policy-*.db"))

records = []

for db_path in db_path_list:
    results = DB(db_path).get_results()
    cost = (
        results.loc[results["prefix"].str.startswith("test")]
        .groupby(["instance_id", "prefix"])["cost"]
        .min()  # min for every problem (of 2 solvers)
        .reset_index()
        .groupby("instance_id")["cost"]
        .median()  # median over 5 runs
        .mean()  # total mean score
    )
    cpu_time = (
        results.loc[
            results["prefix"].str.startswith("config")
            & results["cached"].eq(0)
            & results["surrogate"].eq(0),
            "time",
        ].sum()
        / 3600
    )
    records.append(
        {
            "db_path": str(db_path),
            "cost": cost,
            "cpu_time": cpu_time,
        }
    )

df = pd.DataFrame(records)
df

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0


,db_path,cost,cpu_time
0,C:\Users\grzegorzzakrzewski\Documents\DataScie...,0.23256,1.352241
1,C:\Users\grzegorzzakrzewski\Documents\DataScie...,0.44340,1.458148
2,C:\Users\grzegorzzakrzewski\Documents\DataScie...,0.28644,1.242379
3,C:\Users\grzegorzzakrzewski\Documents\DataScie...,0.42632,0.867457
4,C:\Users\grzegorzzakrzewski\Documents\DataScie...,0.21780,0.907636
5,C:\Users\grzegorzzakrzewski\Documents\DataScie...,0.41612,1.007986
6,C:\Users\grzegorzzakrzewski\Documents\DataScie...,0.19128,0.624734
7,C:\Users\grzegorzzakrzewski\Documents\DataScie...,0.20928,0.624653
8,C:\Users\grzegorzzakrzewski\Documents\DataScie...,0.42932,0.570171
9,C:\Users\grzegorzzakrzewski\Documents\DataScie...,0.44380,0.974133


In [93]:
results.loc[results["prefix"].str.startswith("test"), "cut_off_time"].min()

100.0

In [87]:
df["policy"] = [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]

In [88]:
df.groupby("policy")["cost"].mean()

policy
0    0.320800
1    0.353413
2    0.276627
3    0.374480
4    0.248507
5    0.346613
Name: cost, dtype: float64

In [89]:
df.groupby("policy")["cpu_time"].mean()

policy
0    1.350923
1    0.927693
2    0.606519
3    0.937905
4    1.003690
5    0.704369
Name: cpu_time, dtype: float64

In [4]:
db = DB(db_path)

In [5]:
results = db.get_results()

0.23256